In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pickle
import datetime

In [ ]:
import importlib
import evotsc
import evotsc_plot
import evotsc_core
import evotsc_lib
importlib.reload(evotsc)
importlib.reload(evotsc_lib)
importlib.reload(evotsc_core)
importlib.reload(evotsc_plot)

In [ ]:
## Constants
# Population
nb_indivs = 100
nb_genes = 60

# Genome
intergene = 125
gene_length = 1000
interaction_dist = 5000
interaction_coef = 0.03
sigma_basal = -0.066
sigma_opt = -0.042
epsilon = 0.005
m = 2.5
default_basal_expression = (1 + np.exp(- m)) / 2 # Average of the maximum and minimum expression levels in the model

# Fitness
selection_coef = 250

# Selection
selection_method = "fit-prop" # Choices: "fit-prop", "rank", "exp-rank"

# Environment
sigma_env = 0

# Mutations
inversion_poisson_lam = 2.0
intergene_poisson_lam = 0.0 #2.0
intergene_mutation_var = 0.0 #1e1
basal_sc_mutation_prob = 0.0 #1e-1
basal_sc_mutation_var = 0.0 #1e-4

In [ ]:
rng = np.random.default_rng(seed=424242)

In [ ]:
mutation = evotsc.Mutation(basal_sc_mutation_prob=basal_sc_mutation_prob,
                           basal_sc_mutation_var=basal_sc_mutation_var,
                           intergene_poisson_lam=intergene_poisson_lam,
                           intergene_mutation_var=intergene_mutation_var,
                           inversion_poisson_lam=inversion_poisson_lam)

In [ ]:
init_indiv = (
evotsc_lib.make_random_indiv(intergene=intergene, gene_length=gene_length, nb_genes=nb_genes,
                             default_basal_expression=default_basal_expression,
                             interaction_dist=interaction_dist, interaction_coef=interaction_coef,
                             sigma_basal=sigma_basal, sigma_opt=sigma_opt, epsilon=epsilon, m=m,
                             selection_coef=selection_coef, mutation=mutation, rng=rng))


In [ ]:
fitnesses = 0
for i in range(100):
    indiv = (
evotsc_lib.make_random_indiv(intergene=intergene, gene_length=gene_length, nb_genes=nb_genes,
                             default_basal_expression=default_basal_expression,
                             interaction_dist=interaction_dist, interaction_coef=interaction_coef,
                             sigma_basal=sigma_basal, sigma_opt=sigma_opt, epsilon=epsilon, m=m,
                             selection_coef=selection_coef, mutation=mutation, rng=rng))

    fitnesses += indiv.evaluate(sigma_env)[1]
print(fitnesses/100)

In [ ]:
#evotsc_plot.plot_expr(indiv=init_indiv, sigma_env=0,
#                         plot_title='best generation 0', plot_name=f'best generation {0:05}.pdf')

In [ ]:
evotsc_plot.plot_genome_and_tsc(init_indiv, print_ids=True, sigma=sigma_env, coloring_type='by-id', plot_name=f'best genome gen {0:05}.pdf')

In [ ]:
population = evotsc.Population(init_indiv=init_indiv,
                               nb_indivs=nb_indivs,
                               mutation=mutation,
                               sigma_env=sigma_env,
                               selection_method=selection_method,
                               rng=rng)
population.evaluate()

In [ ]:
step = 50

In [ ]:
for gen in range(1001):    
    
    best_indiv, avg_fit = population.step()

    if gen % 10 == 0:
        time = datetime.datetime.now().strftime("%H:%M:%S")
        print(f'Generation {gen:05} ({time}): best fit {best_indiv.fitness:.5}, avg fit {avg_fit:.5}')
    
    if gen % step == 0:
        #evotsc_plot.plot_expr_AB(indiv=best_indiv, sigma_A=sigma_A, sigma_B=sigma_B,
                                 #plot_title=f'best generation {gen:05}',
                                 #plot_name=f'best generation {gen:05}.pdf')
        #evotsc_plot.explain(best_indiv, sigma_A=sigma_A, sigma_B=sigma_B)
        evotsc_plot.plot_genome_and_tsc(best_indiv, sigma=sigma_env, coloring_type='by-id', plot_name=f'best genome gen {gen:05}.pdf')
        print()
    

In [ ]:
last_indiv, avg = population.step()

In [ ]:
evotsc_plot.explain(last_indiv, sigma_A=sigma_A, sigma_B=sigma_B)

In [ ]:
def run(pop, nb_steps):
    for i in range(nb_steps):
        if i % 10 == 0:
            print(f'Generation {i}')
        pop.step()